# Sleep Stage Transition Analysis

This Jupyter notebook is designed to analyze the simulation output of the thalamocortical network model described in the paper "Cellular and neurochemical basis of sleep stages in the thalamocortical network" by Krishnan et al. The notebook guides you through the process of setting up the environment, running the simulation, loading the output data, and performing various analyses to understand the transitions between different sleep stages.

## Setup and Installation

You can use the pre-installed code on Cybershuttle or you can install the C++ code locally. The simulation code is implemented in C++ and uses OpenMP for parallelism. You will need a C++ compiler and OpenMP support.

## Cloning the Repository

Clone the repository to your local machine using the following command:
```bash
git clone https://github.com/bazhlab-ucsd/sleep-stage-transition.git
```

## Building the Project

Navigate to the project directory and compile the code:
```bash
cd sleep-stage-transition
make network
make run
```


## Running Simulations

The simulation parameters can be modified in the `params.txt` file. Adjust this file to set different levels of neuromodulators and other parameters. Run the simulation with:
```bash
make run
```
# Alternatively you can run the code using Cybershuttle

## Loading and Processing Output Data

After running the simulation, the output files will be saved in the `out` directory. These files contain the membrane voltage and other relevant data of different neuron types.

# For simulations ran through Cybershuttle, the Cybershuttle local agent can transparently fetch these output files